https://python.langchain.com/docs/use_cases/sql/quickstart/

### **Test the sqldb**

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [8]:
llm = ChatOpenAI(temperature =0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm = llm,
    memory = memory,
    verbose = True
)

In [9]:
conversation.predict(input= "hii, remember the secret word is cat")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hii, remember the secret word is cat
AI:


NotFoundError: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}

In [3]:
from langchain_community.utilities import SQLDatabase
from pyprojroot import here
import warnings
warnings.filterwarnings("ignore")

**Connecting to the sqldb**

In [4]:
db_path = str(here("data")) + "/sqldb.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

In [37]:
db

In [38]:
# validate the connection to the vectordb
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

### **Test the access to the environment variables**

In [39]:
from dotenv import load_dotenv
import os
print("Environment variables are loaded:", load_dotenv())
print("test by reading a variable:", os.getenv("OPENAI_API_TYPE"))

Environment variables are loaded: True
test by reading a variable: azure


### **Test your GPT model**

In [40]:
from openai import AzureOpenAI

messages = [
    {"role": "system", "content": str(
        "You are a helpful assistant"
    )},
    {"role": "user", "content": str("hello")}
]

client = AzureOpenAI(
    api_version="2023-03-15-preview",
    azure_endpoint="https://fabric-poc.openai.azure.com/",
    api_key="5a3e842aa1b14dc7b092553422349c8d",
)

response = client.chat.completions.create(
    model= "gpt-4",
    messages=messages
)

print(response.choices[0].message.content)

Hello! How can I assist you today?


### **1. SQL query chain**

In [41]:
from langchain.chat_models import AzureChatOpenAI
import os

model_name = "gpt3-5"
azure_openai_api_key = os.environ["OPENAI_API_KEY"]
azure_openai_endpoint = os.environ["OPENAI_API_BASE"]
llm = AzureChatOpenAI(
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
    azure_deployment="azure",
    model_name=model_name,
    temperature=0.0)

In [42]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many employees are there"})
print(response)

SELECT COUNT(*) FROM Employee


Execute the query to make sure it’s valid

In [43]:
db.run(response)

'[(8,)]'

### **Add QuerySQLDataBaseTool to the chain**
Execute SQL query

**This is the most dangerous part of creating a SQL chain.** Consider carefully if it is OK to run automated queries over your data. Minimize the database connection permissions as much as possible. Consider adding a human approval step to you chains before query execution (see below).

We can use the QuerySQLDatabaseTool to easily add query execution to our chain:

In [44]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

write_query = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)

chain = write_query | execute_query

chain.invoke({"question": "How many employees are there"})

'[(8,)]'

### **Answer the question in a user friendly manner**

In [45]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many employees are there"})

'There are 8 employees.'

### **2. Agents**

Agent which provides a more flexible way of interacting with SQL databases. The main advantages of using the SQL Agent are:

- It can answer questions based on the databases’ schema as well as on the databases’ content (like describing a specific table).
- It can recover from errors by running a generated query, catching the traceback and regenerating it correctly.
- It can answer questions that require multiple dependent queries.
- It will save tokens by only considering the schema from relevant tables.

To initialize the agent, we use create_sql_agent function. This agent contains the SQLDatabaseToolkit which contains tools to:

- Create and execute queries
- Check query syntax
- Retrieve table descriptions
- …

In [6]:
import os
from langchain_openai import AzureChatOpenAI
from langchain.sql_database import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent


# Set the Azure OpenAI environment variables
os.environ["AZURE_OPENAI_API_KEY"] = "5a3e842aa1b14dc7b092553422349c8d"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://fabric-poc.openai.azure.com"
os.environ["AZURE_OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gpt-4"

# Initialize the AzureChatOpenAI instance for Azure
model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)

# Path to your SQLite database
# db_path = str(here("data")) + "/sqldb.db"
# db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

# Create the SQL agent
agent_executor = create_sql_agent(model, db=db, agent_type="openai-tools", verbose=True, handle_parsing_errors=True)



In [53]:
agent_executor.invoke({"input": "Describe the playlisttrack table"})

# agent_executor.invoke("Describe the playlisttrack table")



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'PlaylistTrack'}`



CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/The `PlaylistTrack` table serves to link tracks to playlists, creating a many-to-many relationship between the `Track` and `Playlist` tables. It contains two columns:

- `PlaylistId`: An INTEGER that serves as a foreign key linking to the `Playlist` table. It is part of the primary key for the `PlaylistTrack` table, indicating which playlist a track belongs to.
- `TrackId`: An INTEGER that serves

{'input': 'Describe the playlisttrack table',
 'output': 'The `PlaylistTrack` table serves to link tracks to playlists, creating a many-to-many relationship between the `Track` and `Playlist` tables. It contains two columns:\n\n- `PlaylistId`: An INTEGER that serves as a foreign key linking to the `Playlist` table. It is part of the primary key for the `PlaylistTrack` table, indicating which playlist a track belongs to.\n- `TrackId`: An INTEGER that serves as a foreign key linking to the `Track` table. It is also part of the primary key for the `PlaylistTrack` table, indicating which track is included in the playlist.\n\nBoth `PlaylistId` and `TrackId` are marked as NOT NULL, ensuring that each record in the `PlaylistTrack` table must have valid identifiers for both a playlist and a track. The combination of `PlaylistId` and `TrackId` forms the primary key for the table, meaning that the same track cannot be listed more than once in the same playlist.\n\nHere are a few example rows fro

In [48]:
agent_executor.invoke(
    "List the total sales per country. Which country's customers spent the most?"
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Customer,Invoice,InvoiceLine'}`



CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	Fax	Email	SupportRepId
1	Luís	Gonçalves	Embraer - Empresa Brasileira de Aeronáutica S.A.	Av. Brigadeiro Faria Lima, 21

{'input': "List the total sales per country. Which country's customers spent the most?",
 'output': "The total sales per country are as follows:\n\n- USA: $523.06\n- Canada: $303.96\n- France: $195.1\n- Brazil: $190.1\n- Germany: $156.48\n- United Kingdom: $112.86\n- Czech Republic: $90.24\n- Portugal: $77.24\n- India: $75.26\n- Chile: $46.62\n- Ireland: $45.62\n- Hungary: $45.62\n- Austria: $42.62\n- Finland: $41.62\n- Netherlands: $40.62\n- Norway: $39.62\n- Sweden: $38.62\n- Spain: $37.62\n- Poland: $37.62\n- Italy: $37.62\n- Denmark: $37.62\n- Belgium: $37.62\n- Australia: $37.62\n- Argentina: $37.62\n\nThe country's customers who spent the most are from the USA."}